# Code for running test

In [56]:
from load_datasets import get_aeon_dataset
from pathlib import Path
import os
import torch
import numpy as np

from tsml_eval.experiments import experiments, get_regressor_by_name, run_regression_experiment
from tsml_eval.evaluation.storage import load_regressor_results




def test_regressor(
        regressor, #= TSMLWrapperHydraBoost(),
        regressor_name = "HydraBoost",
    ):
    #get HouseholdPowerConsumption1 dataset
    current_dir = Path(os.path.dirname(os.getcwd()))
    TSER_data_dir = current_dir.parent / "Data" / "TSER"
    dataset_name = "HouseholdPowerConsumption1"
    X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name, TSER_data_dir, "regression")

    #run regression experiment
    run_regression_experiment(
        X_train,
        y_train,
        X_test,
        y_test,
        regressor,
        regressor_name=regressor_name,
        results_path="results/",
        dataset_name=dataset_name,
        resample_id=0,
    )
    rr = load_regressor_results(
        current_dir / "exploring-hydra-boosting" /"results" / regressor_name / "Predictions" / dataset_name / "testResample0.csv"
    )
    print(rr.predictions)
    print(rr.mean_squared_error, "mse")
    print(rr.root_mean_squared_error, "rmse")
    print(rr.mean_absolute_percentage_error, "mape")
    print(rr.r2_score, "r2")
    print(rr.fit_time, "fit time")


# Simple Wrapper no gridsearch

In [57]:
import torch
import pandas as pd
import numpy as np

from sklearn.base import ClassifierMixin, RegressorMixin
from tsml.base import BaseTimeSeriesEstimator

from models.random_feature_representation_boosting import HydraBoost


class TSMLWrapperHydraBoost(RegressorMixin, BaseTimeSeriesEstimator):
    
    def __init__(self, **kwargs):
        super(TSMLWrapperHydraBoost, self).__init__()
        self.hydraboost = HydraBoost(
            n_layers=1,
            init_n_kernels=8,
            init_n_groups=64,
            n_kernels=8,
            n_groups=64,
            max_num_channels=3,
            hydra_batch_size=10000,
            l2_reg=10,
            l2_ghat=0.1,
            boost_lr=1,
            train_top_at = [0, 5, 10],
            **kwargs
        )
        

    def fit(self, X: np.ndarray, y: np.ndarray) -> object:
        """Fit the estimator to training data.

        Parameters
        ----------
        X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
            The training data.
        y : 1D np.ndarray of shape (n_instances)
            The target labels for fitting, indices correspond to instance indices in X

        Returns
        -------
        self :
            Reference to self.
        """
        X = torch.from_numpy(X).float()
        y = torch.from_numpy(y).float()
        y = y.unsqueeze(1)
        self.X_mean = X.mean()
        self.X_std = X.std()
        self.y_mean = y.mean()
        self.y_std = y.std()
        X = (X - self.X_mean) / self.X_std
        y = (y - self.y_mean) / self.y_std
        self.hydraboost.fit(X, y)
        return self


    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predicts labels for sequences in X.

        Parameters
        ----------
        X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
            The training data.

        Returns
        -------
        y : array-like of shape (n_instances)
            Predicted target labels.
        """
        X = torch.from_numpy(X).float()
        X = (X - self.X_mean) / self.X_std
        pred = self.hydraboost(X)
        pred = pred * self.y_std + self.y_mean
        return pred.squeeze().detach().numpy()
        
        

    def _more_tags(self) -> dict:
        return {
            "X_types": ["3darray"],
            "equal_length_only": True,
            "allow_nan": False,
        }


In [55]:
test_regressor(
    regressor = TSMLWrapperHydraBoost(),
    regressor_name = "HydraBoost",
)

training W0
Phi0 shape torch.Size([745, 8192])
[1600.05651855 1238.1640625  1221.96850586 1557.38879395 1824.94348145
 1379.98339844 1697.71594238 1997.15966797 1639.87121582 2560.07348633
 1220.82653809 1848.0871582  1699.41491699 1706.4921875  1342.64855957
 1542.57836914 1316.07849121 1274.05493164 1155.40625     712.20465088
  554.26550293  988.09527588 1601.31396484 2188.98388672 2180.65014648
 1841.53637695 1650.64416504 1891.29345703 2146.93457031 2051.04077148
 2165.74462891 1893.140625   1877.60656738 1825.11474609 1787.85900879
 1318.99133301 2193.06738281 1522.14257812 1709.50732422 1180.23547363
 1108.84350586 1007.70184326 1003.35882568  497.15612793 1082.06518555
 1270.11193848 2275.28491211 1804.5189209  2094.1105957  1546.29125977
 2038.43603516 1813.36242676 1847.92578125 2724.70166016 1399.2956543
 2635.95385742 1925.33862305 2269.30444336 1710.20239258 1338.97827148
 1648.5020752  1235.19799805 1507.59472656 1129.71191406 1129.39685059
 1047.24108887  591.14611816 12

c:\Users\nz423\Code\exploring-hydra-boosting\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Gridsearch Wrapper

In [70]:
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.metrics import roc_auc_score
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable

import numpy as np
import torch
import torch.nn as nn


class SKLearnWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, modelClass=None, **model_params,):
        self.modelClass = modelClass
        self.model_params = model_params
        self.seed = None
        self.model = None
        
        
    def set_params(self, **params):
        self.modelClass = params.pop('modelClass', self.modelClass)
        self.seed = params.pop('seed', self.seed)
        self.model_params.update(params)
        return self


    def get_params(self, deep=True):
        params = {'modelClass': self.modelClass}
        params.update(self.model_params)
        return params
    
    
    def fit(self, X, y):
        if self.seed is not None:
            np.random.seed(self.seed)
            torch.manual_seed(self.seed)
            torch.cuda.manual_seed(self.seed)
        self.model = self.modelClass(**self.model_params)
        self.model.fit(X, y)
        # #classes, either label for binary or one-hot for multiclass
        # if len(y.size()) == 1 or y.size(1) == 1:
        #     self.classes_ = np.unique(y.detach().cpu().numpy())
        # else:
        #     self.classes_ = np.unique(y.argmax(axis=1).detach().cpu().numpy())
        return self


    def predict(self, X):
        return self.model(X).squeeze()#.detach().cpu().squeeze().numpy()
        # #binary classification
        # if len(self.classes_) == 2:
        #     proba_1 = torch.sigmoid(self.model(X))
        #     return (proba_1 > 0.5).detach().cpu().numpy()
        # else:
        #     #multiclass
        #     return torch.argmax(self.model(X), dim=1).detach().cpu().numpy()
    
    # def predict_proba(self, X):
    #     #binary classification
    #     if len(self.classes_) == 2:
    #         proba_1 = torch.nn.functional.sigmoid(self.model(X))
    #         return torch.cat((1 - proba_1, proba_1), dim=1).detach().cpu().numpy()
    #     else:
    #         #multiclass
    #         logits = self.model(X)
    #         proba = torch.nn.functional.softmax(logits, dim=1)
    #         return proba.detach().cpu().numpy()
    
    # def decision_function(self, X):
    #     logits = self.model(X)
    #     return logits.detach().cpu().numpy()


    
    # def score(self, X, y):
    #     logits = self.model(X)
    #     if y.size(1) == 1:
    #         y_true = y.detach().cpu().numpy()
    #         y_score = logits.detach().cpu().numpy()
    #         auc = roc_auc_score(y_true, y_score)
    #         return auc
    #     else:
    #         pred = torch.argmax(logits, dim=1)
    #         y = torch.argmax(y, dim=1)
    #         acc = (pred == y).float().mean()
    #         return acc.detach().cpu().item()
    
    
    
class TSMLGridSearchWrapper(RegressorMixin, BaseTimeSeriesEstimator):
    
    def __init__(self,
                 holdour_or_kfold: Literal["holdout", "kfold"] = "kfold",
                 kfolds: Optional[int] = 5,
                 holdout_percentage: Optional[float] = 0.2,
                 seed: Optional[int] = None,
                 modelClass=None, 
                 model_param_grid: Dict[str, List[Any]] = {}
        ):
        self.holdour_or_kfold = holdour_or_kfold
        self.kfolds = kfolds
        self.holdout_percentage = holdout_percentage
        self.seed = seed
        self.modelClass = modelClass
        self.model_param_grid = model_param_grid
        super(TSMLGridSearchWrapper, self).__init__()
        

    def fit(self, X: np.ndarray, y: np.ndarray) -> object:
        """Fit the estimator to training data, with gridsearch hyperparameter optimization
        on holdout or kfold cross-validation.

        Parameters
        ----------
        X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
            The training data.
        y : 1D np.ndarray of shape (n_instances)
            The target labels for fitting, indices correspond to instance indices in X

        Returns
        -------
        self :
            Reference to self.
        """
        # TODO regression only
        X = torch.from_numpy(X).float()
        y = torch.from_numpy(y).float()
        y = y.unsqueeze(1)
        self.X_mean = X.mean()
        self.X_std = X.std()
        self.y_mean = y.mean()
        self.y_std = y.std()
        X = (X - self.X_mean) / self.X_std
        y = (y - self.y_mean) / self.y_std
        
        # Configure cross validation
        if self.holdour_or_kfold == "kfold":
            cv = KFold(n_splits=self.kfolds, shuffle=True, random_state=self.seed)
        else:  # holdout
            cv = ShuffleSplit(n_splits=1, test_size=self.holdout_percentage, random_state=self.seed)
                
        # Perform grid search
        self.grid_search = GridSearchCV(
            estimator=SKLearnWrapper(modelClass=self.modelClass),
            param_grid={**self.model_param_grid, "seed": [self.seed]},
            cv=cv,
            scoring="neg_mean_squared_error", # TODO regression only???
        )
        self.grid_search.fit(X, y)

        # Store best model
        self.best_model = self.grid_search.best_estimator_
        self.best_params = self.grid_search.best_params_
        print("self.best_params", self.best_params)
        return self
        
        
    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predicts labels for sequences in X.

        Parameters
        ----------
        X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
            The training data.

        Returns
        -------
        y : array-like of shape (n_instances)
            Predicted target labels.
        """
        X = torch.from_numpy(X).float()
        X = (X - self.X_mean) / self.X_std
        pred = self.best_model.predict(X) #TODO regression only?
        pred = pred * self.y_std + self.y_mean
        return pred.squeeze().detach().cpu().numpy()
        

    def _more_tags(self) -> dict:
        return {
            "X_types": ["3darray"],
            "equal_length_only": True,
            "allow_nan": False,
        }

In [ ]:
test_regressor(
    TSMLGridSearchWrapper(
        "kfold",
        kfolds=3,
        seed=0,
        modelClass=HydraBoost,
        model_param_grid={
            "n_layers": [1],              # [0,1,3,6,10] ?
            "init_n_kernels": [8],
            "init_n_groups": [64],
            "n_kernels": [8],
            "n_groups": [64],
            "max_num_channels": [3],
            "hydra_batch_size": [10000],
            "l2_reg": [10],                # [0.0001, 0.001, 0.01, 0.1, 1] ?
            "l2_ghat": [0.01, 1],          # [0.0001, 0.001, 0.01, 0.1, 1] ?
            "boost_lr": [1],
            "train_top_at": [[0, 5, 10]],
        },
    ),
regressor_name = "HydraBoostGridSearch",
)

training W0
Phi0 shape torch.Size([496, 8192])
training W0
Phi0 shape torch.Size([497, 8192])
training W0
Phi0 shape torch.Size([497, 8192])
training W0
Phi0 shape torch.Size([496, 8192])
training W0
Phi0 shape torch.Size([497, 8192])
training W0
Phi0 shape torch.Size([497, 8192])
training W0
Phi0 shape torch.Size([745, 8192])
self.best_params {'boost_lr': 1, 'hydra_batch_size': 10000, 'init_n_groups': 64, 'init_n_kernels': 8, 'l2_ghat': 0.01, 'l2_reg': 10, 'max_num_channels': 3, 'n_groups': 64, 'n_kernels': 8, 'n_layers': 1, 'seed': 0, 'train_top_at': [0, 5, 10]}
[1642.73950195 1339.05163574 1205.52783203 1630.63696289 1818.23083496
 1494.93847656 1640.51171875 1933.3046875  1617.32409668 2562.12304688
 1346.82409668 1899.80639648 1716.19604492 1637.89196777 1442.05383301
 1544.9864502  1229.63378906 1350.53833008 1082.3449707   690.6918335
  638.54089355  955.67822266 1526.09423828 2236.4152832  2277.37353516
 1902.18896484 1800.43994141 1742.06860352 2168.48999023 1951.28857422
 211

c:\Users\nz423\Code\exploring-hydra-boosting\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# TSMLOptunaWrapper